In [1]:
import pandas as pd
import plotly.express as px
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [29]:
def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)
    
     # Remove NAN Value
    df.dropna(inplace=True)
    
     # Drop features with high null counts
    df.drop(columns=["ocean_proximity"], inplace=True)
    
    return df

In [30]:
df = wrangle(r"C:\Users\sanus\Desktop\DS\web\housing.csv")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20433 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
dtypes: float64(9)
memory usage: 1.6 MB
None


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


We spent some time in the last lesson zooming in on a useful subset of the SCF, and this time, we're going to zoom in even further. One of the persistent issues we've had with this dataset is that it includes some outliers in the form of ultra-wealthy households. This didn't make much of a difference for our last analysis, but it could pose a problem in this lesson, so we're going to focus on families with net worth under $2 million.

Calculate the variance for all the features in df, and create a Series top_ten_var with the 10 features with the largest variance.

In [31]:
df.var().sort_values().tail(10)

median_income         3.607307e+00
longitude             4.014324e+00
latitude              4.563981e+00
housing_median_age    1.585536e+02
households            1.461527e+05
total_bedrooms        1.775654e+05
population            1.284161e+06
total_rooms           4.775403e+06
median_house_value    1.332539e+10
dtype: float64

In [32]:
# Calculate variance, get 10 largest features
top_ten_var = df.var().sort_values().tail(10)
top_ten_var

median_income         3.607307e+00
longitude             4.014324e+00
latitude              4.563981e+00
housing_median_age    1.585536e+02
households            1.461527e+05
total_bedrooms        1.775654e+05
population            1.284161e+06
total_rooms           4.775403e+06
median_house_value    1.332539e+10
dtype: float64

In [33]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_var,
    y=top_ten_var.index,
    title=" High Variance Features"
)

fig.update_layout(xaxis_title="Variance", yaxis_title="Feature")

fig.show()

One thing that we've seen throughout this project is that many of the wealth indicators are highly skewed, with a few outlier households having enormous wealth. Those outliers can affect our measure of variance. Let's see if that's the case with one of the features from top_five_var.

In [34]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame=df,
    x= "median_house_value",
    title = "Distribution of median_house_value"
)

fig.update_layout(xaxis_title="Value [$]")

fig.show()

In [35]:
df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            20630, 20631, 20632, 20633, 20634, 20635, 20636, 20637, 20638,
            20639],
           dtype='int64', length=20433)


Whoa! The dataset is massively right-skewed because of the huge outliers on the right side of the distribution. Even though we already excluded households with a high net worth with our wrangle function, the variance is still being distorted by some extreme outliers.

The best way to deal with this is to look at the trimmed variance, where we remove extreme values before calculating variance. We can do this using the trimmed_variance function from the SciPy library.

Calculate the trimmed variance for the features in df. Your calculations should not include the top and bottom 10% of observations. Then create a Series top_ten_trim_var with the 10 features with the largest variance.

In [23]:
trimmed_var(df["median_house_value"])

5550558673.845247

In [38]:
top_ten_trim_var = df.apply(trimmed_var, limits=(0.1, 0.1)).sort_values()
top_ten_trim_var

median_income         1.191597e+00
longitude             2.887484e+00
latitude              3.041485e+00
housing_median_age    8.272390e+01
households            2.873237e+04
total_bedrooms        3.410124e+04
population            2.422167e+05
total_rooms           7.997655e+05
median_house_value    5.550559e+09
dtype: float64

In [39]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x=top_ten_trim_var,
    y=top_ten_trim_var.index,
    title=" High Variance Features"
)

fig.update_layout(xaxis_title="Trimmed Variance", yaxis_title="Feature")

fig.show()

In [43]:
high_var_cols = top_ten_trim_var.head(8).index.to_list()
high_var_cols

['median_income',
 'longitude',
 'latitude',
 'housing_median_age',
 'households',
 'total_bedrooms',
 'population',
 'total_rooms']

# Split

In [50]:
X = df[high_var_cols]
print("X shape:", X.shape)
X.head()

X shape: (20433, 8)


,median_income,longitude,latitude,housing_median_age,households,total_bedrooms,population,total_rooms
0,8.3252,-122.23,37.88,41.0,126.0,129.0,322.0,880.0
1,8.3014,-122.22,37.86,21.0,1138.0,1106.0,2401.0,7099.0
2,7.2574,-122.24,37.85,52.0,177.0,190.0,496.0,1467.0
3,5.6431,-122.25,37.85,52.0,219.0,235.0,558.0,1274.0
4,3.8462,-122.25,37.85,52.0,259.0,280.0,565.0,1627.0


# Build Model
# Iterate

During our EDA, we saw that we had a scale issue among our features. That issue can make it harder to cluster the data, so we'll need to fix that to help our analysis along. One strategy we can use is standardization, a statistical method for putting all the variables in a dataset on the same scale. Let's explore how that works here. Later, we'll incorporate it into our model pipeline.

Create a DataFrame X_summary with the mean and standard deviation for all the features in X

In [51]:
X.mean()

median_income            3.871162
longitude             -119.570689
latitude                35.633221
housing_median_age      28.633094
households             499.433465
total_bedrooms         537.870553
population            1424.946949
total_rooms           2636.504233
dtype: float64

In [52]:
X_summary = X.aggregate(["mean", "std"]).astype(int)
X_summary

,median_income,longitude,latitude,housing_median_age,households,total_bedrooms,population,total_rooms
mean,3,-119,35,28,499,537,1424,2636
std,1,2,2,12,382,421,1133,2185


Create a StandardScaler transformer, use it to transform the data in X, and then put the transformed data into a DataFrame named X_scaled.

In [53]:
x_scaled = (X - X.mean()) / X.std()
print("mean:", round(x_scaled.mean()))
print("std:", round(x_scaled.std()))

mean: median_income        -0.0
longitude            -0.0
latitude             -0.0
housing_median_age    0.0
households           -0.0
total_bedrooms       -0.0
population           -0.0
total_rooms           0.0
dtype: float64
std: median_income         1.0
longitude             1.0
latitude              1.0
housing_median_age    1.0
households            1.0
total_bedrooms        1.0
population            1.0
total_rooms           1.0
dtype: float64


In [54]:
# Instantiate transformer
ss = StandardScaler()

# Transform `X`
X_scaled_data = ss.fit_transform(X)

# Put `X_scaled_data` into DataFrame
X_scaled = pd.DataFrame(X_scaled_data, columns=X.columns)

print("X_scaled shape:", X_scaled.shape)
X_scaled.head()

X_scaled shape: (20433, 8)


,median_income,longitude,latitude,housing_median_age,households,total_bedrooms,population,total_rooms
0,2.345163,-1.327314,1.051717,0.982163,-0.976833,-0.970325,-0.973320,-0.803813
1,2.332632,-1.322323,1.042355,-0.606210,1.670373,1.348276,0.861339,2.042130
2,1.782939,-1.332305,1.037674,1.855769,-0.843427,-0.825561,-0.819769,-0.535189
3,0.932970,-1.337296,1.037674,1.855769,-0.733562,-0.718768,-0.765056,-0.623510
4,-0.013143,-1.337296,1.037674,1.855769,-0.628930,-0.611974,-0.758879,-0.461970


In [55]:
X_scaled_summary = X_scaled.aggregate(["mean", "std"]).astype(int)
X_scaled_summary

,median_income,longitude,latitude,housing_median_age,households,total_bedrooms,population,total_rooms
mean,0,0,0,0,0,0,0,0
std,1,1,1,1,1,1,1,1


And that's what it should look like. Remember, standardization takes all the features and scales them so that they all have a mean of 0 and a standard deviation of 1.

Now that we can compare all our data on the same scale, we can start making clusters. Just like we did last time, we need to figure out how many clusters we should have

Use a for loop to build and train a K-Means model where n_clusters ranges from 2 to 12 (inclusive). Your model should include a StandardScaler. Each time a model is trained, calculate the inertia and add it to the list inertia_errors, then calculate the silhouette score and add it to the list silhouette_scores.

In [56]:
n_clusters = range(2, 13)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
    # Build model
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=k, random_state=42))
    # Train model
    model.fit(X)
    # Calculate inertia
    inertia_errors.append(model.named_steps["kmeans"].inertia_)
    # Calculate silhouette score
    silhouette_scores.append(silhouette_score(X, model.named_steps["kmeans"].labels_))

print("Inertia:", inertia_errors[:3])
print()
print("Silhouette Scores:", silhouette_scores[:3])

Inertia: [120080.83588795051, 88539.65992442876, 75805.41466455598]

Silhouette Scores: [0.6739271128441698, 0.028339954702048253, 0.05661629980904151]


Just like last time, let's create an elbow plot to see how many clusters we should use

Use plotly express to create a line plot that shows the values of inertia_errors as a function of n_clusters. Be sure to label your x-axis "Number of Clusters", your y-axis "Inertia", and use the title "K-Means Model: Inertia vs Number of Clusters"

In [57]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=inertia_errors, title="K-Means Model: Inertia vs Number of Clusters"
)

fig.update_layout(xaxis_title="Number of Clusters (k)", yaxis_title="Inertia")

fig.show()

You can see that the line starts to flatten out around 4 or 5 clusters

Let's make another line plot based on the silhouette scores.

Use plotly express to create a line plot that shows the values of silhouette_scores as a function of n_clusters. Be sure to label your x-axis "Number of Clusters", your y-axis "Silhouette Score", and use the title "K-Means Model: Silhouette Score vs Number of Clusters".

In [58]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=silhouette_scores, title="K-Means Model: Silhouette Score vs Number of Clusters"
)

fig.update_layout(xaxis_title="Number of Clusters (k)", yaxis_title="Silhouette Score")

fig.show()

This one's a little less straightforward, but we can see that the best silhouette scores occur when there are 3 or 4 clusters.

Putting the information from this plot together with our inertia plot, it seems like the best setting for n_clusters will be 3.

Build and train a new k-means model named final_model. Use the information you gained from the two plots above to set an appropriate value for the n_clusters argument. Once you've built and trained your model, submit it to the grader for evaluation.

In [59]:
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=3, random_state=42)
)
final_model.fit(X)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=3, random_state=42))])

# Communicate

In [60]:
# Extract the labels that your final_model created during training and assign them to the variable labels
labels = final_model.named_steps["kmeans"].labels_
print(labels[:5])

[0 2 0 0 0]


In [61]:
# Create a DataFrame xgb that contains the mean values of the features in X for each of the clusters in your final_model
xgb = X.groupby(labels).mean()
xgb

,median_income,longitude,latitude,housing_median_age,households,total_bedrooms,population,total_rooms
0,3.770380,-121.720466,37.964415,29.773015,405.350603,436.620947,1107.123245,2191.739471
1,3.891868,-118.021090,33.943441,29.631339,425.843773,454.479306,1268.000559,2167.589765
2,4.226869,-119.160386,35.249269,16.625604,1433.457126,1570.225242,3986.471014,7836.036836


Use plotly express to create a side-by-side bar chart from `xgb` that shows the mean of the features in `X` for each of the clusters in your `final_model`. Be sure to label the x-axis `"Cluster"`, the y-axis `"Value [$]"`, and use the title `"Mean Household Finances by Cluster"

In [62]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode="group",
    title="Mean Household Finances by Cluster"
)

fig.update_layout(xaxis_title="Cluster", yaxis_title="Value [$]")

fig.show()

reate a PCA transformer, use it to reduce the dimensionality of the data in X to 2, and then put the transformed data into a DataFrame named X_pca. The columns of X_pca should be named "PC1" and "PC2".

In [63]:
# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns=["PC1", "PC2"])

print("X_pca shape:", X_pca.shape)
X_pca.head()

X_pca shape: (20433, 2)


,PC1,PC2
0,-2130.812078,-253.038337
1,4528.873803,-1033.067252
2,-1523.104358,-352.451023
3,-1653.973260,-204.720872
4,-1326.585665,-349.814030


So there we go: our five dimensions have been reduced to two. Let's make a scatter plot and see what we get.

Use plotly express to create a scatter plot of X_pca using seaborn. Be sure to color the data points using the labels generated by your final_model. Label the x-axis "PC1", the y-axis "PC2", and use the title "PCA Representation of Clusters"

In [64]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame=X_pca,
    x="PC1",
    y="PC2",
    color=labels.astype(str),
    title="PCA Representation of Clusters"
    
)
fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")
fig.show()

One limitation of this plot is that it's hard to explain what the axes here represent. In fact, both of them are a combination of the five features we originally had in X, which means this is pretty abstract. Still, it's the best way we have to show as much information as possible as an explanatory tool for people outside the data science community.

So what does this graph mean? It means that we made four tightly-grouped clusters that share some key features. If we were presenting this to a group of stakeholders, it might be useful to show this graph first as a kind of warm-up, since most people understand how a two-dimensional object works. Then we could move on to a more nuanced analysis of the data.

Just something to keep in mind as you continue your data science journey